# BIFROST reduction of simulated data

This notebook demonstrates the basic data reduction workflow for BIFROST.
It uses data that was simulated with McStas and a dedicated workflow that can process McStas data.

In [ ]:
import scipp as sc
import sciline
from ess import bifrost
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon, tof_lookup_table_simulation
from ess.spectroscopy.types import *
import scippnexus as snx

from ess.spectroscopy.types import SampleRun

BIFROST NeXus files store detector data in 45 separate NXdetector groups, one per detector triplet.
For the time being, we need to specify the names of these NXdetector groups when creating the workflow.
So load them from the input file:

In [ ]:
with snx.File(simulated_elastic_incoherent_with_phonon()) as f:
    detector_names = list(f['entry/instrument'][snx.NXdetector])

Generally, we would use all triplets, but for this example, we only use the first two.
This reduces the size of the data and the time to compute it.

In [ ]:
detector_names = detector_names[:2]

Next, construct the workflow which is a [sciline.Pipeline](https://scipp.github.io/sciline/generated/classes/sciline.Pipeline.html) and encodes the entire reduction procedure.
We need to provide a couple of parameters so we can run the workflow:

In [ ]:
workflow = bifrost.BifrostSimulationWorkflow(detector_names)
# Set the input file name:
workflow[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()
# Set the lookup table for frame unwrapping:
workflow[TimeOfFlightLookupTable] = sc.io.load_hdf5(tof_lookup_table_simulation())
# We need to read many objects from the file,
# keeping it open improves performance: (optional)
workflow[PreopenNeXusFile] = PreopenNeXusFile(True)

Next, draw the workflow as a graph to inspect the steps it will take to reduce the data.
Note the groups where entries are labeled with `triplet=x`. (These labels will be `dim_0=x` if you don't have [Pandas](https://pandas.pydata.org/) installed.)
In this example, there are only two triplets but, as explained above, in a realistic case, there would be 45.

In [ ]:
workflow.visualize(EnergyData[SampleRun], graph_attr={"rankdir": "LR"})

We are ready to compute the reduced data.
We use the naive scheduler of sciline because it tends to perform better for BIFROST than the [Dask](https://docs.dask.org/en/stable/index.html) scheduler.
But this is optional.

In [ ]:
scheduler = sciline.scheduler.NaiveScheduler()
data = workflow.compute(EnergyData[SampleRun], scheduler=scheduler)

The result contains coordinates for the sample table and detector rotation angles `a3` and `a4`, respectively.
It also contains event coordinates for `energy_transfer` and two momentum transfers, one in the lab frame and one in the sample table frame.

In [ ]:
data

We can plot the counts as a function of energy transfer and $a_3$ by removing the unused dimensions.
As expected, it is independent of $a_3$.

In [ ]:
(
    data['a4', 0]
    .bins.concat(['triplet', 'tube', 'length'])
    .hist(energy_transfer=sc.linspace('energy_transfer', -0.05, 0.05, 200, unit='meV'))
).plot()

We can also plot the counts as a function of the momentum transfer in the sample table frame $Q$.
For this, we first need to create a 2D slice of $Q$.
For simplicity, we use the x and z axes (see https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html for definitions).
But we could use any other normalised, orthogonal vectors in the dot products.
The plot is a bit coarse because we only used 2 triplets.

In [ ]:
d = data['a4', 0].bins.concat().copy()
x = sc.vector([1, 0, 0])
z = sc.vector([0, 0, 1])
d.bins.coords['Qx'] = sc.dot(x, d.bins.coords['sample_table_momentum_transfer'])
d.bins.coords['Qz'] = sc.dot(z, d.bins.coords['sample_table_momentum_transfer'])
d.hist(Qz=200, Qx=200).plot(norm='log')